In [ ]:
!pip install openvino
!pip install onnx
# the torch require onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 52.8 MB/s eta 0:00:00


In [ ]:
import datetime
import time


In [ ]:

import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
from openvino.runtime import Core


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Original model training
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')

print('Finished Training')


# Original model testing
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


def test_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


accuracy = test_model(model, testloader)
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')





100%|██████████| 9912422/9912422 [00:00<00:00, 80933677.85it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 94415973.36it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28665956.76it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18247632.92it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch 1, Loss: 0.8080143455439793
Epoch 2, Loss: 0.17717020529340197
Epoch 3, Loss: 0.11696892441101452
Epoch 4, Loss: 0.09210410513190319
Epoch 5, Loss: 0.07588499082155081
Epoch 6, Loss: 0.065385498448279
Epoch 7, Loss: 0.057819277322642616
Epoch 8, Loss: 0.051873547301294086
Epoch 9, Loss: 0.04678121987886147
Epoch 10, Loss: 0.04331380693933992
Finished Training
Accuracy of the model on the 10000 test images: 98.82 %
Model has been converted to ONNX format and saved as simple_model.onnx


In [ ]:
###### The following is openvino part


dummy_input = torch.randn(64, 1, 28, 28)



torch.onnx.export(model, dummy_input, "simple_model.onnx", export_params=True,
                          opset_version=11, do_constant_folding=True,
                          input_names=['input'], output_names=['output'],
                          dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})
print("Model has been converted to ONNX format and saved as simple_model.onnx")




ie = Core()

model_path = "simple_model.onnx"
ovmodel = ie.read_model(model=model_path)

# Compile the model for a specific device
cuta = time.time()
compiled_ovmodel = ie.compile_model(model=ovmodel, device_name="CPU")
cutb = time.time()
print("compile time:",cutb-cuta)
#input_layer = compiled_ovmodel.input(0)
#output_layer = compiled_ovmodel.output(0)



Model has been converted to ONNX format and saved as simple_model.onnx
compile time: 0.017530441284179688


In [ ]:
model.eval()
correct = 0
total = 0
ori_t_s=0
for data in testloader:
    images, labels = data
    ori_t_a=time.time()
    outputs = model(images)
    ori_t_b=time.time()
    ori_t_s= ori_t_s + ori_t_b - ori_t_a

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print(f'Accuracy of the model on the 10000 test images: {accuracy} %')
print("exact time for original model: ",ori_t_s)



correct = 0
total = 0
vino_t_s=0
for data in testloader:
    images, labels = data
    vino_t_a=time.time()
    outputs = compiled_ovmodel(images)[compiled_ovmodel.output(0)]
    vino_t_b=time.time()
    vino_t_s= vino_t_s + vino_t_b - vino_t_a
    outputs = torch.from_numpy(outputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the vinomodel on the 10000 test images: {accuracy} %')
print("exact time for vino model: ",vino_t_s)

Accuracy of the model on the 10000 test images: 98.82 %
exact time for original model:  2.717346429824829
Accuracy of the vinomodel on the 10000 test images: 98.82 %
exact time for vino model:  1.9785869121551514
